In [ ]:
import sys
import pandas as pd
import spacy
from textblob import TextBlob
from textblob import Word
import en_core_web_md
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize,sent_tokenize
from autocorrect import Speller
spell = Speller(lang='en')
import string
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
nlp = spacy.load("en_core_web_md")

MASUKKAN DATA ULASAN & DATA DAFTAR KEBUTUHAN

In [ ]:
print_ulasan = open("Ulasan-MarketPlace.txt", "r", encoding='utf-8')
print_req = open("Req-Comm.txt", "r", encoding='utf-8')
ulasan = print_ulasan.read()
requirement = print_req.read()

In [ ]:
#print(ulasan)
print(ulasan)
#x = ulasan.split('.')
#print("\n\n")
#print(x)

#print(req)
print(requirement)
r = requirement.replace('+','\n')
print("\n\n")
print(r)

PRAPROSES DATA REQUIREMENT

In [ ]:
req = r.translate(str.maketrans('', '', string.punctuation))
#1.Case Folding
req_n = spell(req).lower()
#print ('\n'+req_n)

#2.Tokenisasi
req_t = req_n.split('\n')
#print (ulasan_t)

#3.Hapus Stopwords
list_req=[]
stop_words = nltk.corpus.stopwords.words('english')
newStopWords = ['user']
stop_words.extend(newStopWords)
for i in range(len(req_t)):
    token = nltk.word_tokenize(req_t[i])
    filtered_sentence = [w for w in token if not w in stop_words]
    filtered_sentence = []

    for w in token: 
        if w not in stop_words: 
            filtered_sentence.append(w+" ")
    string_fs = ""
    string_fs = string_fs.join(filtered_sentence)
    list_req.append(string_fs)
print(list_req)
#print(req_t)

PRAPOSES DATA ULASAN

In [ ]:
#1.Pembersihan Data
import re
#hapus kelebihan titik
ulasan_c =ulasan.replace("....",".")
ulasan_c =ulasan_c.replace("...",".")
ulasan_c =ulasan_c.replace("..",".")
ulasan_c =ulasan_c.replace('"',"")
ulasan_c =ulasan_c.replace("\n"," ")
ulasan_c =ulasan_c.replace("won't","will not")
ulasan_c =ulasan_c.replace("app ","application ")
ulasan_c =ulasan_c.replace("n't"," not")
ulasan_c =re.sub('(?<=\d).(?=\d)', ',', ulasan_c)
ulasan_c =ulasan_c.replace(".",". ")
print (ulasan_c)

#hapus emoji
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
ulasan_c=deEmojify(ulasan_c)
#print('ulasan c \n'+ulasan_c)

#2.Spelling Normalization
ulasan_n = spell(ulasan_c)
print('Spelling Normalization')
print ('ulasan n \n'+ulasan_n)

#3.Tokenisasi
ulasan_t = sent_tokenize(ulasan_n)
print('Tokenisasi')
print (ulasan_t)

#4.POS Tagging
list_ulasan=[]
for i in range(len(ulasan_t)):
    token = nltk.word_tokenize(ulasan_t[i])
    list_ulasan.append(nltk.pos_tag(token))

print('\n+POS Tagging')
print(list_ulasan)

ANALISIS SENTIMEN

In [ ]:
df = pd.DataFrame(columns=['Teks','Polarity','Subjectivity','SF'])
for i in range(len(ulasan_t)):
    ulasanTB= TextBlob(ulasan_t[i])
    polarity = ulasanTB.sentiment.polarity
    subjectivity = ulasanTB.sentiment.subjectivity
    df.loc[i, ['Teks']] = ulasan_t[i]
    df.loc[i, ['Polarity']] = polarity
    df.loc[i, ['Subjectivity']] = subjectivity
df
df.to_excel(r'SENTIMEN-Commerce.xlsx')

POS TAG CHUNKING

In [ ]:
def converttostr(input_seq, seperator):
   # Join all the strings in list
   final_str = seperator.join(input_seq)
   return final_str
grammar = ('''PN:{<PRP\$|PRP>+}
NC:{(<DT>*<JJ|JJS|JJR>*)?(<IN>*<NN|NNP|NNS|NNPS>)*} 
VERB:{<VB|VBG|VBZ|VBD|VBN|VBP>+<IN>*} 
NP:{((<NC>+<,>)*<NC>*<CC>+)*<NC|IN>+}
SF: {(<VERB|(JJ|JJS|JJR)|NP>+<DT>*<NP|(RB|RBR|RBS)|(JJ|JJS|JJR)>+)+}
    ''')

chunkParser = nltk.RegexpParser(grammar)

for i in range(len(list_ulasan)):
    chunkList=[]
    print(str(i)+'.'+str(ulasan_t[i]))
    tree = chunkParser.parse(list_ulasan[i])
    #tree.draw()
    for j in tree.subtrees(filter=lambda x: x.label() == 'SF'):
        print (j)
        #print (get_word(j))
        text=' '.join([w for w, t in j.leaves()])
        chunkList.append(text)
    print(chunkList)
    df.loc[i, ['SF']] = converttostr(chunkList, ',')

df

df_new=(df.set_index(['Teks','Polarity','Subjectivity'])
   .apply(lambda x: x.str.split(',').explode())
   .reset_index())


In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 

def lemmatize_text(text):
    output=""
    for sentence in text:
        output=(" ".join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]))
    return output

def stopwords_removal(text):
    filtered_sentence=""
    for sentence in text:
        filtered_sentence = (" ".join([w for w in w_tokenizer.tokenize(text) if not w in stop_words]))
    return filtered_sentence

df_ekstraksi = df_new
df_ekstraksi['SF'] = df_ekstraksi['SF'].str.lower()
df_ekstraksi['SF'] = df_ekstraksi.SF.apply(lemmatize_text)
df_ekstraksi['SF'] = df_ekstraksi.SF.apply(stopwords_removal)
df_group=df_ekstraksi
df_ekstraksi = df_ekstraksi.groupby('SF').agg({'Polarity':['min','max','mean'],'Subjectivity':['min','max','mean']}).reset_index()

group=df_group.groupby(['SF'])['Teks'].count().reset_index()
df_ekstraksi['Count'] = group['Teks']
df_ekstraksi=df_ekstraksi.sort_values('Count',ascending=False)
df_ekstraksi=df_ekstraksi[df_ekstraksi.SF != '']
df_ekstraksi['SF'] = df_ekstraksi['SF'].str.replace('^ +| +$','_')
df_ekstraksi['SF'] = df_ekstraksi['SF'].str.replace('_','')
df_ekstraksi=df_ekstraksi[df_ekstraksi['SF'].str.contains(" ")]
df_ekstraksi=df_ekstraksi[df_ekstraksi['SF'].str.contains(" ")]
df_ekstraksi=df_ekstraksi.reset_index(drop=True)
df_ekstraksi

SIMILARITY

In [ ]:
import numpy as np

In [ ]:
for i in range(len(list_req)):
    df_ekstraksi[str(i+1)] = np.nan
    for j in range(len(df_ekstraksi)):
        #print('req='+str(i)+"||"+'ulasan='+str(j))
        position=str(i+1)
        token_req=nlp(list_req[i])
        token_ulasan=nlp(df_ekstraksi.loc[j,'SF'].to_string())
        df_ekstraksi.loc[j, [position]]=token_req.similarity(token_ulasan)
df_ekstraksi

In [ ]:
df_ekstraksi.to_excel(r'Similarity-MarketPlace-.xlsx')

FILTER AND REPORT

In [ ]:
import xlsxwriter
writer = pd.ExcelWriter('SF-MarketPlace-New.xlsx',engine='xlsxwriter')

cell_format = workbook.add_format({'align': 'center',
                                   'valign': 'vcenter',
                                   'bold': True})
for i in range(len(list_req)):
    position=str(i+1)
    print(position)
    df_filter = df_ekstraksi[df_ekstraksi[str(i+1)]>=0.5]
    df_filter= df_filter[['SF','Polarity','Subjectivity','Count',position]].sort_values(by=position,ascending=False)
    df_filter.rename(columns={position: 'Similarity'}, inplace=True)
    df_filter= df_filter.reset_index(drop=True)
    df_filter.set_index(['SF'])
    df_filter.to_excel(writer, sheet_name=position, startrow=1, index=True) 
    worksheet = writer.sheets[position]
    worksheet.merge_range('B1:J1',str(req_t[i].upper()),cell_format)
    #worksheet.write_string(0, 1, str(req_t[i]))
    #worksheet.write_string('A1', str(req_t[i]))
writer.save()